1. Calculation of Kalman filter steps

I first show all the calculations I did and then I
will present the full filled table.

The initial belief is:

\begin{equation}
\mu_0 = 2, \Sigma_0 = 5
\end{equation}


Since the using system's dynamic model is constant:
\begin{equation}
\bar{\mu}_1 = \mu_0 = 2, 
\end{equation}



